## Reshaping a Pandas Dataframe: Wide-to-Long for Drought Characteristics data

The input data came from ArcGIS ZonalStats output then concatenating all the csv into single file in wide format</br>
This script is to reshape the table into long format so it fit with Stata-style csv requirements.

In [1]:
import pandas as pd

In [2]:
# Load the data
duration_df = pd.read_csv("wld_cli_spei12_gt20_duration_1958_2021_terraclimate_05deg.csv")
duration12_df = pd.read_csv("wld_cli_spei12_gt20_duration12m_1958_2021_terraclimate_05deg.csv")
event_df = pd.read_csv("wld_cli_spei12_gt20_event_1958_2021_terraclimate_05deg.csv")
intensity_df = pd.read_csv("wld_cli_spei12_gt20_event_1958_2021_terraclimate_05deg.csv")
magnitude_df = pd.read_csv("wld_cli_spei12_gt20_event_1958_2021_terraclimate_05deg.csv")

In [3]:
# Melt the data frames to long format
duration_long = pd.melt(duration_df, id_vars=["lat", "lon", "orig_id"], var_name="date", value_name="duration")
duration12_long = pd.melt(duration12_df, id_vars=["lat", "lon", "orig_id"], var_name="date", value_name="duration12")
event_long = pd.melt(event_df, id_vars=["lat", "lon", "orig_id"], var_name="date", value_name="event")
intensity_long = pd.melt(intensity_df, id_vars=["lat", "lon", "orig_id"], var_name="date", value_name="intensity")
magnitude_long = pd.melt(magnitude_df, id_vars=["lat", "lon", "orig_id"], var_name="date", value_name="magnitude")

In [4]:
# Merge the data frames
df = duration_long.merge(duration12_long, on=["lat", "lon", "orig_id", "date"])
df = df.merge(event_long, on=["lat", "lon", "orig_id", "date"])
df = df.merge(intensity_long, on=["lat", "lon", "orig_id", "date"])
df = df.merge(magnitude_long, on=["lat", "lon", "orig_id", "date"])

In [5]:
# Extract year and month from the date column
df["year"] = df["date"].str[:4]
df["month"] = df["date"].str[4:6]

In [6]:
# Drop the date column
df = df.drop("date", axis=1)

In [7]:
# Rearrange the columns to the desired order
df = df[["lat", "lon", "orig_id", "year", "month", "duration", "duration12", "event", "intensity", "magnitude"]]

In [8]:
# Save the resulting data frame to a Stata-style CSV file
df.to_csv("wld_cli_spei12_gt20_monthly_droughtcharacteristics_1958_2021_terraclimate_05deg.csv", index=False)

In [9]:
# Check the result
check1 = pd.read_csv("wld_cli_spei12_gt20_monthly_droughtcharacteristics_1958_2021_terraclimate_05deg.csv")
check1

,lat,lon,orig_id,year,month,duration,duration12,event,intensity,magnitude
0,-55.75,-69.75,1,1958,12,0,0,0,0,0
1,-55.75,-69.25,2,1958,12,0,0,0,0,0
2,-55.75,-68.75,3,1958,12,0,0,0,0,0
3,-55.75,-68.25,4,1958,12,0,0,0,0,0
4,-55.75,-67.75,5,1958,12,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
52638747,79.25,94.75,70057,2021,12,0,0,0,0,0
52638748,79.75,94.75,70058,2021,12,0,0,0,0,0
52638749,80.25,94.75,70059,2021,12,0,0,0,0,0
52638750,80.75,94.75,70060,2021,12,0,0,0,0,0


In [10]:
# Group the data by year and get the mean value for each year
df_year = df.groupby(["lat", "lon", "orig_id", "year"]).mean()

In [11]:
# Reset the index to obtain a flat data frame
df_year = df_year.reset_index()

In [12]:
# Save the resulting data frame to a CSV file
df_year.to_csv("wld_cli_spei12_gt20_annual_droughtcharacteristics_1958_2021_terraclimate_05deg.csv", index=False)

In [13]:
# Check the result
check2 = pd.read_csv("wld_cli_spei12_gt20_annual_droughtcharacteristics_1958_2021_terraclimate_05deg.csv")
check2

,lat,lon,orig_id,year,duration,duration12,event,intensity,magnitude
0,-55.75,-69.75,1,1958,0.000000,0.0,0.000000,0.000000,0.000000
1,-55.75,-69.75,1,1959,0.000000,0.0,0.000000,0.000000,0.000000
2,-55.75,-69.75,1,1960,0.000000,0.0,0.000000,0.000000,0.000000
3,-55.75,-69.75,1,1961,0.000000,0.0,0.000000,0.000000,0.000000
4,-55.75,-69.75,1,1962,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
4450299,83.25,-24.25,38336,2017,0.416667,0.0,0.166667,0.166667,0.166667
4450300,83.25,-24.25,38336,2018,0.000000,0.0,0.000000,0.000000,0.000000
4450301,83.25,-24.25,38336,2019,0.000000,0.0,0.000000,0.000000,0.000000
4450302,83.25,-24.25,38336,2020,0.000000,0.0,0.000000,0.000000,0.000000
